# Import libraries

In [1]:
import pandas as pd
import numpy as np
import os
import config
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from tqdm import tqdm
from GMHI import GMHI

# Load data

In [2]:
X = pd.read_csv(os.path.join(config.CLEAN_DIR, "taxonomic_features.csv"), index_col=[0, 1])
y = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.csv"), index_col=[0, 1]).iloc[:, [0]]
display(X.head())
display(y.head())

k__Archaea  k__Archaea|p__Euryarchaeota  \
Study_ID Sample Accession                                            
GMHI-10  SAMN03283239        0.000000                     0.000000   
         SAMN03283266        0.000000                     0.000000   
         SAMN03283281        0.009764                     0.009764   
         SAMN03283245        0.106067                     0.106067   
         SAMN03283294        0.000000                     0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria  \
Study_ID Sample Accession                                                   
GMHI-10  SAMN03283239                                            0.000000   
         SAMN03283266                                            0.000000   
         SAMN03283281                                            0.009764   
         SAMN03283245                                            0.106067   
         SAMN03283294                                            0.000000   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales  \
Study_ID Sample Accession                                                                         
GMHI-10  SAMN03283239                                               0.000000                      
         SAMN03283266                                               0.000000                      
         SAMN03283281                                               0.009764                      
         SAMN03283245                                               0.106067                      
         SAMN03283294                                               0.000000                      

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae  \
Study_ID Sample Accession                                                                                                
GMHI-10  SAMN03283239                                               0.000000                                             
         SAMN03283266                                               0.000000                                             
         SAMN03283281                                               0.009764                                             
         SAMN03283245                                               0.106067                                             
         SAMN03283294                                               0.000000                                             

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter  \
Study_ID Sample Accession                                                                                                                      
GMHI-10  SAMN03283239                                               0.000000                                                                   
         SAMN03283266                                               0.000000                                                                   
         SAMN03283281                                               0.009764                                                                   
         SAMN03283245                                               0.072453                                                                   
         SAMN03283294                                               0.000000                                                                   

                           k__Archaea|p__Euryarchaeota|c__Methanobacteria|o__Methanobacteriales|f__Methanobacteriaceae|g__Methanobrevibacter|s__Methanobrevibacter_smithii  \
Study_ID Sample Accession                                                                                                                                                    
GMHI-10  SAMN03283239                                               0.000000              

PHENOTYPE:Healthy_Nonhealthy
Study_ID Sample Accession                              
GMHI-10  SAMN03283239                              True
         SAMN03283266                              True
         SAMN03283281                              True
         SAMN03283245                              True
         SAMN03283294                              True

# Generate group splits

In [3]:
# indentify groups to partition dataset

sample_studies = np.array(X.index.get_level_values(0))
o = OrdinalEncoder()
groups = o.fit_transform(sample_studies.reshape((len(sample_studies), 1))).flatten()
logo = LeaveOneGroupOut()

# Generate predictions in leave-one-study-out cross validation!

## GMHI

In [4]:
# instantiate the gmhi model
gmhi = GMHI(use_shannon=True, theta_f=1.4, theta_d=0.1)

# use only non-viral species
X_GMHI = X.copy()
X_GMHI = X_GMHI[list(filter(lambda x : "s__" in x and "virus" not in x, X_GMHI.columns))]

# generate predictions for each left out study (doing this manually because GMHI class required pandas dataframes)
GMHI_scores_ISV = np.zeros(y.shape)
for test_study in tqdm(np.unique(X.index.get_level_values(0))):
    train_studies = ~X_GMHI.index.get_level_values(0).isin([test_study])
    X_train = X_GMHI.loc[train_studies]
    y_train = y.loc[train_studies]
    X_test = X_GMHI.loc[test_study]
    gmhi.fit(X_train, y_train)
    GMHI_scores_ISV[X.index.get_level_values(0) == test_study] = np.expand_dims(gmhi.decision_function(X_test), -1)
    
GMHI_scores_ISV = pd.DataFrame(GMHI_scores_ISV, index=y.index, columns=["GMHI_ISV"])

100%|███████████████████████████████████████████████████████████████| 57/57 [00:10<00:00,  5.69it/s]


In [5]:
print(balanced_accuracy_score(y, GMHI_scores_ISV > 0))
idx = (abs(GMHI_scores_ISV) > 0.1).values
print(balanced_accuracy_score(y[idx], GMHI_scores_ISV[idx] > 0))
idx = (abs(GMHI_scores_ISV) > 1).values
print(balanced_accuracy_score(y[idx], GMHI_scores_ISV[idx] > 0))

0.6982397476995704
0.7029333962495481
0.763789661067015


## GMHI2

In [6]:
# instantiate the logistic regression classifier (gmhi2)
gmhi2 = LogisticRegression(random_state=42, penalty="l1", solver="liblinear", C=config.REGULARIZATION, class_weight="balanced")

# generate logit predictions for each left out study
GMHI2_scores_ISV = cross_val_predict(gmhi2, X > config.PRESENCE_CUTOFF, y.values.flatten(), method="decision_function",
    groups=groups, cv=logo, verbose=2, n_jobs=-1
)

GMHI2_scores_ISV = pd.DataFrame(GMHI2_scores_ISV, index=y.index, columns=["GMHI2_ISV"])
GMHI2_scores_ISV

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  55 out of  57 | elapsed:   15.8s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  57 out of  57 | elapsed:   15.8s finished


GMHI2_ISV
Study_ID Sample Accession           
GMHI-10  SAMN03283239       1.876379
         SAMN03283266       0.273884
         SAMN03283281       0.942891
         SAMN03283245       1.152237
         SAMN03283294       2.140906
...                              ...
P98      SAMEA6104852       0.581640
         SAMEA6104851      -0.988054
         SAMEA6104850       0.817093
         SAMEA6104885      -0.135115
         SAMEA6104858       0.887893

[8869 rows x 1 columns]

In [7]:
print(balanced_accuracy_score(y, GMHI2_scores_ISV > 0))
idx = (abs(GMHI2_scores_ISV) > 0.1).values
print(balanced_accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))
idx = (abs(GMHI2_scores_ISV) > 1).values
print(balanced_accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))

0.7414930320660262
0.754163778358591
0.8571608110724518


In [8]:
print(accuracy_score(y, GMHI2_scores_ISV > 0))
idx = (abs(GMHI2_scores_ISV) > 0.1).values
print(accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))
idx = (abs(GMHI2_scores_ISV) > 1).values
print(accuracy_score(y[idx], GMHI2_scores_ISV[idx] > 0))

0.7401059871462398
0.7525760985431719
0.8568521031207599


In [9]:
# conf matrix

print(((GMHI2_scores_ISV > 0).values & ~y.values).sum())
print(((GMHI2_scores_ISV < 0).values & ~y.values).sum())
print(((GMHI2_scores_ISV > 0).values & y.values).sum())
print(((GMHI2_scores_ISV < 0).values & y.values).sum())

790
2321
4242
1515


# Save cv scores

In [10]:
GMHI_scores_ISV.to_csv(os.path.join(config.LOG_DIR, "GMHI_scores_ISV.csv"))
GMHI2_scores_ISV.to_csv(os.path.join(config.LOG_DIR, "GMHI2_scores_ISV.csv"))